In [ ]:
# imports
import numpy as np
import pandas as pd
import json

In [ ]:
countries_id_df = pd.read_csv('countries_id.csv')
country_map = countries_id_df.set_index('to_country_id')['to_country'].to_dict()

votes_df = pd.read_csv('votes.csv')
votes_df['to_country'] = votes_df['to_country'].map(country_map)
votes_df['from_country'] = votes_df['from_country'].map(country_map)

In [ ]:
incoming_votes_df = votes_df[votes_df['round'] == 'final'].drop(columns=['round', 'from_country_id', 'from_country', 'to_country_id'])
incoming_votes_df = incoming_votes_df.groupby(['year', 'to_country']).sum().fillna(0)
incoming_votes_df

In [ ]:
contestants_df = pd.read_csv('contestants.csv')[['year', 'to_country', 'song', 'performer']]
contestants_df

In [ ]:
top_countries_df = incoming_votes_df.sort_values(['total_points'], ascending=False).groupby('year').head(5)
top_countries_df = top_countries_df.sort_index()
top_countries_df['pos'] = top_countries_df.sort_values(['total_points'], ascending=False).groupby(['year']).cumcount() + 1
top_countries_df = top_countries_df.drop(columns=['tele_points', 'jury_points', 'total_points'])
top_countries_df = top_countries_df.reset_index()
top_countries_df

In [ ]:
result_df = pd.merge(top_countries_df, contestants_df, on=['year', 'to_country'], how='inner')
result_df['country'] = result_df['to_country']
result_df = result_df.drop(columns='to_country')
result_df = result_df.set_index(['year', 'country']).drop_duplicates()
result_df.tail(10)

In [ ]:
result_json = {}

for outer_key, group_df in result_df.groupby(level=0):
    inner = group_df.droplevel(0)
    result_json[outer_key] = inner.to_dict(orient="index")

In [ ]:
with open("top_5_data.json", "w") as f:
    json.dump(result_json, f, indent=4)